In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
import time
from datetime import datetime

from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in lat_lngs:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

634

In [4]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jamestown
Processing Record 2 of Set 1 | bredasdorp
Processing Record 3 of Set 1 | coquimbo
Processing Record 4 of Set 1 | pochutla
Processing Record 5 of Set 1 | vanimo
Processing Record 6 of Set 1 | lorengau
Processing Record 7 of Set 1 | punta arenas
Processing Record 8 of Set 1 | qaanaaq
Processing Record 9 of Set 1 | nikolskoye
Processing Record 10 of Set 1 | lasa
Processing Record 11 of Set 1 | busselton
Processing Record 12 of Set 1 | adrar
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | tabou
Processing Record 15 of Set 1 | salalah
Processing Record 16 of Set 1 | mahajanga
Processing Record 17 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 18 of Set 1 | ribeira grande
Processing Record 19 of Set 1 | dukat
Processing Record 20 of Set 1 | maryville
Processing Record 21 of Set 1 | hobart
Processing Record 22 of Set 1 | upernavik
Processing Record 2

In [5]:
len(city_data)

577

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,24.12,68,90,5.99,US,2022-01-03 04:48:35
1,Bredasdorp,-34.5322,20.0403,58.28,88,83,1.14,ZA,2022-01-03 04:48:35
2,Coquimbo,-29.9533,-71.3436,60.53,94,0,0.00,CL,2022-01-03 04:48:36
3,Pochutla,15.7432,-96.4661,76.82,74,0,4.05,MX,2022-01-03 04:48:36
4,Vanimo,-2.6741,141.3028,78.71,90,100,8.70,PG,2022-01-03 04:48:36
5,Lorengau,-2.0226,147.2712,81.45,80,100,13.78,PG,2022-01-03 04:48:37
6,Punta Arenas,-53.1500,-70.9167,41.11,65,75,9.22,CL,2022-01-03 04:45:31
7,Qaanaaq,77.4840,-69.3632,-3.32,68,58,7.54,GL,2022-01-03 04:48:37
8,Nikolskoye,59.7035,30.7861,19.40,95,99,7.18,RU,2022-01-03 04:48:38
9,Lasa,46.6166,10.7002,41.88,79,38,5.99,IT,2022-01-03 04:48:39


In [10]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]

city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Jamestown,US,2022-01-03 04:48:35,42.0970,-79.2353,24.12,68,90,5.99
1,Bredasdorp,ZA,2022-01-03 04:48:35,-34.5322,20.0403,58.28,88,83,1.14
2,Coquimbo,CL,2022-01-03 04:48:36,-29.9533,-71.3436,60.53,94,0,0.00
3,Pochutla,MX,2022-01-03 04:48:36,15.7432,-96.4661,76.82,74,0,4.05
4,Vanimo,PG,2022-01-03 04:48:36,-2.6741,141.3028,78.71,90,100,8.70
...,...,...,...,...,...,...,...,...,...
572,Douala,CM,2022-01-03 05:04:49,4.0483,9.7043,76.78,89,59,3.51
573,Aklavik,CA,2022-01-03 05:04:49,68.2191,-135.0107,-17.75,94,100,8.59
574,Constitucion,CL,2022-01-03 05:04:49,-35.3333,-72.4167,59.02,93,3,5.68
575,Sioux Lookout,CA,2022-01-03 05:04:50,50.1001,-91.9170,-3.89,93,81,4.85


In [13]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")